In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings("ignore")

In [2]:
prev = pd.read_excel("/Users/avery/OneDrive/Documents/GitHub/Clinical_TLB_2023-2024/lung_cancer_tlb.xlsx")
update = pd.read_excel('/Users/avery/OneDrive/Documents/publication_meta_data.xlsx')
merged = pd.merge(prev, update, left_on='pub_id', right_on="Patient")
df = merged.drop(["CancerType", "sample_id", "pub_id", "Patient"], axis = 1)

In [3]:
# get location of cut off values
lower_column_index = df.columns.get_loc("T51")
upper_column_index = df.columns.get_loc("T83.1")
label_column_index = df.columns.get_loc("Diagnosis")
cancer_column_index = df.columns.get_loc("Stage")
column_indices = np.arange(lower_column_index, upper_column_index)
column_indices = np.append(column_indices, cancer_column_index)
column_indices = np.append(column_indices, label_column_index)

df = df.iloc[:, column_indices]

In [4]:
cancer_type = "AC"
prediction_df = df.loc[df["Diagnosis"] == cancer_type]
prediction_df = prediction_df.drop(["Diagnosis"], axis = 1)
prediction_df["Stage"] = np.where(prediction_df["Stage"] == "Early", 0, 1)


In [11]:
indices = np.arange(7)

prediction_df.iloc[indices, :]

,T51,T51.1,T51.2,T51.3,T51.4,T51.5,T51.6,T51.7,T51.8,T51.9,...,T82.2,T82.3,T82.4,T82.5,T82.6,T82.7,T82.8,T82.9,T83,Stage
3,0.02349,0.02299,0.02220,0.02148,0.02112,0.02066,0.02038,0.01988,0.01932,0.01888,...,0.01373,0.01314,0.01223,0.01144,0.01087,0.01040,0.00961,0.00858,0.00776,0
7,0.01754,0.01720,0.01723,0.01764,0.01761,0.01704,0.01611,0.01553,0.01511,0.01473,...,0.02722,0.02697,0.02659,0.02636,0.02622,0.02594,0.02544,0.02482,0.02418,0
8,0.02066,0.02019,0.02033,0.02050,0.02040,0.02034,0.02014,0.01979,0.01945,0.01918,...,0.01536,0.01599,0.01544,0.01440,0.01312,0.01209,0.01060,0.00928,0.00835,0
9,0.01920,0.01920,0.01906,0.01890,0.01859,0.01820,0.01769,0.01713,0.01622,0.01543,...,0.03076,0.03009,0.02921,0.02830,0.02697,0.02576,0.02474,0.02354,0.02205,1
11,0.01592,0.01554,0.01546,0.01519,0.01457,0.01397,0.01365,0.01380,0.01382,0.01341,...,0.02110,0.02069,0.02018,0.01946,0.01899,0.01905,0.01848,0.01771,0.01722,1
22,0.01670,0.01695,0.01710,0.01714,0.01710,0.01698,0.01681,0.01657,0.01631,0.01599,...,0.01859,0.01757,0.01657,0.01572,0.01492,0.01417,0.01344,0.01274,0.01204,0
24,0.01036,0.01056,0.01072,0.01092,0.01111,0.01130,0.01148,0.01162,0.01169,0.01173,...,0.01618,0.01563,0.01507,0.01448,0.01390,0.01328,0.01266,0.01204,0.01145,0


In [5]:
performance_metrics = pd.DataFrame(columns=['Weighted Accuracy', 'AUC', 'n_estimators', "max_depth", "max_features"])

# Set number of bootstraps
total_bootstraps = 1000

# Length of df
num_rows = prediction_df.shape[0]

# Create array of all indices in the full data set
all_indices = np.arange(num_rows)

counter = 0

# Columns to drop
#drop_cols = ['sample_id', 'pub_id', 'CancerType']

# Loop for specified iterations
for i in range(total_bootstraps):
    
    # Randomly select indices to use as the train set
    train_indices = np.random.choice(num_rows, num_rows, replace=True)

    # Get the train set using the indices
    train_set = prediction_df.iloc[train_indices, : ]

    # Get indices not included in train_indices to use as the test set
    test_indices = np.setdiff1d(all_indices, train_indices)

    # Get the test set using test indices
    test_set = prediction_df.iloc[test_indices, :]

    while(len(np.unique(train_set['Stage'])) < 2 or len(np.unique(test_set['Stage'])) < 2):

            # generate new train and test sets
            train_indices = np.random.choice(num_rows, num_rows, replace=True)

            # Get the train set using the indices
            train_set = prediction_df.iloc[train_indices, : ]

            # Get indices not included in train_indices to use as the test set
            test_indices = np.setdiff1d(all_indices, train_indices)

            # Get the test set using test indices
            test_set = prediction_df.iloc[test_indices, :]

            counter+=1

    # Drop 'CancerType' column from train and test sets
    train_set_no_target = train_set.drop('Stage', axis=1)
    test_set_no_target = test_set.drop('Stage', axis=1)

    for trees in [100, 250, 500, 1000]:
            
        for depth in [num_rows//2, None]:

            for features in ["sqrt", "log2", None]:

                # Initialize random forest (default settings)
                clf = RandomForestClassifier(n_estimators=trees, max_depth=depth, max_features=features, n_jobs= -1)

                # Train the random forest
                clf = clf.fit(train_set_no_target, train_set['Stage'])

                # Get probabilities
                test_probabilities = clf.predict_proba(test_set_no_target)

                # Test the decision tree
                test_predictions = clf.predict(test_set_no_target)

                # Calculate weighted accuracy
                balanced_acc = balanced_accuracy_score(test_set['Stage'], test_predictions)

                # Calculate AUC
                auc = roc_auc_score(test_set['Stage'], test_probabilities[:, 1])

                # Append accuracy, AUC to results DataFrame
                performance_metrics.loc[len(performance_metrics)] = [balanced_acc, auc, trees, depth, features]

    #performance_metrics.to_excel('AC_Stage_Results.xlsx', index=False)

    print(i)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [10]:
performance_metrics = pd.DataFrame(columns=['Weighted Accuracy', 'AUC', 'n_estimators', "max_depth", "max_features"])

# Set number of bootstraps
total_bootstraps = 1000

# Length of df
num_rows = prediction_df.shape[0]

# Create array of all indices in the full data set
all_indices = np.arange(num_rows)

counter = 0

# Columns to drop
#drop_cols = ['sample_id', 'pub_id', 'CancerType']

# Loop for specified iterations
for i in range(total_bootstraps):
    
    # Randomly select indices to use as the train set
    train_indices = np.random.choice(num_rows, num_rows, replace=True)

    # Get the train set using the indices
    train_set = prediction_df.iloc[train_indices, : ]

    # Get indices not included in train_indices to use as the test set
    test_indices = np.setdiff1d(all_indices, train_indices)

    # Get the test set using test indices
    test_set = prediction_df.iloc[test_indices, :]

    while(len(np.unique(train_set['Stage'])) < 2 or len(np.unique(test_set['Stage'])) < 2):

            # generate new train and test sets
            train_indices = np.random.choice(num_rows, num_rows, replace=True)

            # Get the train set using the indices
            train_set = prediction_df.iloc[train_indices, : ]

            # Get indices not included in train_indices to use as the test set
            test_indices = np.setdiff1d(all_indices, train_indices)

            # Get the test set using test indices
            test_set = prediction_df.iloc[test_indices, :]

            counter+=1

    # Drop 'CancerType' column from train and test sets
    train_set_no_target = train_set.drop('Stage', axis=1)
    test_set_no_target = test_set.drop('Stage', axis=1)

    for trees in [100, 250, 500, 1000]:
            
        for depth in [num_rows//2, None]:

            for features in ["sqrt", "log2", None]:

                # Initialize random forest (default settings)
                clf = RandomForestClassifier(n_estimators=trees, max_depth=depth, max_features=features, n_jobs= -1)

                # Train the random forest
                clf = clf.fit(train_set_no_target, train_set['Stage'])

                # Get probabilities
                test_probabilities = clf.predict_proba(test_set_no_target)

                # Test the decision tree
                test_predictions = clf.predict(test_set_no_target)

                # Calculate weighted accuracy
                balanced_acc = balanced_accuracy_score(test_set['Stage'], test_predictions)

                # Calculate AUC
                auc = roc_auc_score(test_set['Stage'], test_probabilities[:, 1])

                # Append accuracy, AUC to results DataFrame
                performance_metrics.loc[len(performance_metrics)] = [balanced_acc, auc, trees, depth, features]

    #performance_metrics.to_excel('AC_Stage_Results.xlsx', index=False)

    print(i)


KeyboardInterrupt: 